In [1]:
# Change directory to level up to import functions
import os
os.chdir('..')

In [ ]:
## Summary
# Stay away from the 5.0 mids, get a 5.0 mid from another team
# 6.5 fwds are worth a punt if nailed to start
# No significant difference at other price points or positions, by different team strength

In [2]:
import pandas as pd
from src.data_prep.analysis import load_combine_fpl_data

In [36]:
import pandas as pd
import scipy.stats as stats

season_years = ['2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24']
df = load_combine_fpl_data(season_years=season_years)
df_subset = df[df['team_strength'] <=5 ]

# Function to filter the DataFrame based on position and value_first_gw
def filter_data(df, position, value):
    return df[(df['position'] == position) & (df['value_first_gw'] == value)]

# Function to perform the t-test and calculate statistics for a filtered DataFrame
def perform_ttest(filtered_df):
    # Group by 'promoted_from_championship' column
    group1 = filtered_df[filtered_df['promoted_from_championship'] == 1]['total_points']
    group2 = filtered_df[filtered_df['promoted_from_championship'] == 0]['total_points']

    # Check if both groups have enough data to perform the t-test
    if len(group1) > 0 and len(group2) > 0:
        # Calculate statistics for each group
        avg_promoted = group1.mean()
        avg_not_promoted = group2.mean()
        sample_size_promoted = len(group1)
        sample_size_not_promoted = len(group2)

        # Perform independent t-test
        t_stat, p_value = stats.ttest_ind(group1, group2, equal_var=False)  # Welch's t-test

        return {
            'sample_size_promoted': sample_size_promoted,
            'sample_size_not_promoted': sample_size_not_promoted,
            'average_score_promoted': avg_promoted,
            'average_score_not_promoted': avg_not_promoted,
            't_test': t_stat,
            'p_value': p_value
        }
    else:
        # Return None if insufficient data
        return None

# Function to loop through unique position and value_first_gw combinations
def loop_combinations(df):
    results = []
    
    unique_positions = df['position'].unique()
    unique_values = df['value_first_gw'].unique()

    for position in unique_positions:
        for value in unique_values:
            # Filter data for the current combination
            filtered_df = filter_data(df, position, value)

            # Perform the t-test and get results
            result = perform_ttest(filtered_df)
            
            if result:  # Only append if there is valid t-test data
                result.update({
                    'position': position,
                    'value_first_gw': value
                })
                results.append(result)
    
    return pd.DataFrame(results)

# Main function that calls the looping function and returns a DataFrame with all results
def perform_ttest_on_df(df_subset):
    return loop_combinations(df_subset)

# Example usage:
result_df = perform_ttest_on_df(df_subset)
result_df.sort_values(['position','value_first_gw'])


c:\Users\edwar\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_stats_py.py:1113: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
c:\Users\edwar\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_stats_py.py:1113: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


,sample_size_promoted,sample_size_not_promoted,average_score_promoted,average_score_not_promoted,t_test,p_value,position,value_first_gw
12,11,18,52.727273,73.722222,-1.796724,0.088021,DEF,40
10,61,263,73.377049,74.133080,-0.171431,0.864327,DEF,45
11,5,176,79.000000,84.931818,-0.358628,0.737254,DEF,50
7,1,3,70.000000,83.333333,NaN,NaN,FWD,45
9,4,7,85.500000,97.142857,-0.660045,0.526372,FWD,50
6,10,18,94.900000,75.833333,1.141625,0.277388,FWD,55
5,6,26,101.333333,97.500000,0.254106,0.805841,FWD,60
8,4,29,127.000000,102.103448,2.500787,0.047650,FWD,65
14,1,7,81.000000,80.714286,NaN,NaN,GK,40
13,16,54,108.625000,102.425926,0.637845,0.529166,GK,45


In [ ]:
### Let's look at the 6.5 FWDs

In [29]:
df_top_players = df_subset[(df_subset['position']=='FWD') & (df_subset['value_first_gw']==65)& (df_subset['promoted_from_championship']==1)]
df_top_players=df_top_players[['name','team','total_points','goals_scored','season']]
df_top_players

,name,team,total_points,goals_scored,season
525,Aleksandar Mitrovic,Fulham,133,11,2018-19
758,Teemu Pukki,Norwich,135,11,2019-20
1175,Ivan Toney,Brentford,138,12,2021-22
1452,Aleksandar Mitrović,Fulham,102,13,2022-23


In [42]:
### 
promoted_from_championship = 0
df_top_players = df_subset[(df_subset['position']=='FWD') & (df_subset['value_first_gw']==65)& (df_subset['promoted_from_championship']==promoted_from_championship)]
df_top_players=df_top_players[['name','team','total_points','goals_scored','season']]
df_top_players.sort_values('total_points',ascending=False)

,name,team,total_points,goals_scored,season
1593,Dominic Solanke,Bournemouth,174,19,2023-24
1606,Julián Álvarez,Man City,142,9,2023-24
517,Joshua King,Bournemouth,140,12,2018-19
761,Chris Wood,Burnley,132,14,2019-20
999,Callum Wilson,Newcastle,130,12,2020-21
46,Álvaro Negredo,Middlesbrough,126,9,2016-17
532,Ayoze Pérez,Newcastle,124,10,2018-19
52,Fernando Llorente,Swansea,121,12,2016-17
544,Chris Wood,Burnley,118,9,2018-19
1017,Michail Antonio,West Ham,114,10,2020-21


In [34]:
path = 'data/championship_goals/2021-2022.csv'
pd.read_csv(path).head(10)

,Player,Country,Team,Goals,Season
0,Aleksandar Mitrović,Serbia,Fulham FC,43,2021-2022
1,Dominic Solanke,England,AFC Bournemouth,29,2021-2022
2,Ben Brereton,Chile,Blackburn Rovers,22,2021-2022
3,Joël Piroe,Netherlands,Swansea City,22,2021-2022
4,Andreas Weimann,Austria,Bristol City,22,2021-2022
5,Karlan Grant,England,West Bromwich Albion,18,2021-2022
6,Viktor Gyökeres,Sweden,Coventry City,17,2021-2022
7,Elijah Adebayo,England,Luton Town,16,2021-2022
8,Brennan Johnson,Wales,Nottingham Forest,16,2021-2022
9,Emil Riis,Denmark,Preston North End,16,2021-2022


In [9]:
import pandas as pd
import scipy.stats as stats
import numpy as np

# Function to round down values to the nearest 10
def round_down(value):
    return (value // 10) * 10

# Function to filter the DataFrame based on position and value_first_gw
def filter_data(df, position, value):
    return df[(df['position'] == position) & (df['rounded_value_first_gw'] == value)]

# Function to perform the t-test and calculate statistics for a filtered DataFrame
def perform_ttest(filtered_df):
    # Group by 'promoted_from_championship' column
    group1 = filtered_df[filtered_df['promoted_from_championship'] == 1]['total_points']
    group2 = filtered_df[filtered_df['promoted_from_championship'] == 0]['total_points']

    # Check if both groups have enough data to perform the t-test
    if len(group1) > 0 and len(group2) > 0:
        # Calculate statistics for each group
        avg_promoted = group1.mean()
        avg_not_promoted = group2.mean()
        sample_size_promoted = len(group1)
        sample_size_not_promoted = len(group2)

        # Perform independent t-test
        t_stat, p_value = stats.ttest_ind(group1, group2, equal_var=False)  # Welch's t-test

        return {
            'sample_size_promoted': sample_size_promoted,
            'sample_size_not_promoted': sample_size_not_promoted,
            'average_score_promoted': avg_promoted,
            'average_score_not_promoted': avg_not_promoted,
            't_test': t_stat,
            'p_value': p_value
        }
    else:
        # Return None if insufficient data
        return None

# Function to loop through unique position and value_first_gw combinations
def loop_combinations(df):
    results = []
    
    unique_positions = df['position'].unique()
    unique_values = df['rounded_value_first_gw'].unique()

    for position in unique_positions:
        for value in unique_values:
            # Filter data for the current combination
            filtered_df = filter_data(df, position, value)

            # Perform the t-test and get results
            result = perform_ttest(filtered_df)
            
            if result:  # Only append if there is valid t-test data
                result.update({
                    'position': position,
                    'value_first_gw': value
                })
                results.append(result)
    
    return pd.DataFrame(results)

# Main function that prepares the DataFrame and calls the looping function
def perform_ttest_on_df(df_subset, round_values=False):
    # Round values down to the nearest 10 if required
    if round_values:
        df_subset['rounded_value_first_gw'] = df_subset['value_first_gw'].apply(round_down)
    else:
        df_subset['rounded_value_first_gw'] = df_subset['value_first_gw']

    return loop_combinations(df_subset)

# Example usage:
# Assuming df_subset is your input DataFrame
result_df = perform_ttest_on_df(df_subset, round_values=True)
result_df.sort_values(['position', 'value_first_gw'])


C:\Users\edwar\AppData\Local\Temp\ipykernel_15460\1074558581.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['rounded_value_first_gw'] = df_subset['value_first_gw'].apply(round_down)
c:\Users\edwar\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_stats_py.py:1113: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
c:\Users\edwar\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_stats_py.py:1113: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


,sample_size_promoted,sample_size_not_promoted,average_score_promoted,average_score_not_promoted,t_test,p_value,position,value_first_gw
7,66,218,73.212121,76.160550,-0.683856,0.495900,DEF,40
6,5,149,79.000000,84.765101,-0.348933,0.744080,DEF,50
5,1,3,70.000000,83.333333,NaN,NaN,FWD,40
4,10,19,104.200000,89.210526,0.943093,0.363214,FWD,50
3,9,48,113.888889,102.520833,1.030760,0.324075,FWD,60
8,17,50,107.000000,109.740000,-0.298507,0.767679,GK,40
2,13,49,75.230769,68.938776,0.817000,0.425877,MID,40
1,42,216,79.142857,84.550926,-1.226852,0.224295,MID,50
0,8,84,97.500000,105.142857,-0.565677,0.587098,MID,60


In [8]:
# Example usage:
filter_by_position=False
# result_df = perform_ttest_on_df(df_subset, filter_by_position=filter_by_position)  # With position
result_df = perform_ttest_on_df(df_subset, filter_by_position=filter_by_position)  # Without position
result_df.sort_values(['value_first_gw', 'position'] if filter_by_position else ['value_first_gw'])

,sample_size_promoted,sample_size_not_promoted,average_score_promoted,average_score_not_promoted,t_test,p_value,value_first_gw
4,9,12,56.000000,79.250000,-1.806024,0.093250,40
1,70,161,80.471429,82.459627,-0.439197,0.661344,45
2,24,138,78.708333,89.884058,-2.088506,0.042804,50
5,21,91,90.952381,88.494505,0.287373,0.776089,55
0,10,53,103.800000,99.358491,0.354808,0.729286,60
3,3,34,125.000000,113.000000,0.921857,0.420136,65
